In [1]:
import spot
import buddy
spot.setup()
from spot.jupyter import display_inline

ModuleNotFoundError: No module named 'buddy'

In [92]:
def trail_to_automaton(trail, bdddict):
    aut = spot.make_twa_graph(bdddict)
    k = len(trail)
    aut.new_states(k)
    for i, step in enumerate(trail):
        cond = buddy.bddtrue
        for (ap, val) in step:
            if val == 0: 
                continue
            var = aut.register_ap(ap)
            if val == 1:
                cond &= buddy.bdd_ithvar(var)
            else:
                cond &= buddy.bdd_nithvar(var)
        
        if i != k - 1:
            aut.new_edge(i, i+1, cond)
        else:
            #for j in range(k):
            aut.new_edge(i, 1, cond)
    return aut

In [93]:
def trail_to_automaton_all(trail, bdddict):
    aut = spot.make_twa_graph(bdddict)
    k = len(trail)
    id_sub_state  = dict()
    id_last_states = dict()
    cond_id = [buddy.bddtrue for i in range(k)]
    curr_state = 0
    aut.new_states(k + sum([i for i in range(1,k+1)]))
    for i, step in enumerate(trail):
        cond = buddy.bddtrue
        for (ap, val) in step:
            if val == 0: 
                continue
            var = aut.register_ap(ap)
            if val == 1:
                cond &= buddy.bdd_ithvar(var)
            else:
                cond &= buddy.bdd_nithvar(var)
        cond_id[i] = cond
        if i != k - 1:
            aut.new_edge(i, i+1, cond)
        else:
            for z,j in zip(range(k),range(k-1,-1,-1)):
                curr_state+=1
                aut.new_edge(i, curr_state, cond)
                begin_state = curr_state
                end_state   = curr_state + j
                id_last_states[end_state] = z
                #automata of one state (loop = 0)
                if begin_state == end_state:
                    aut.new_edge(curr_state, curr_state, cond)
                    id_sub_state[curr_state] = (z,k - 1)

                #automata of at least two states (loop > 0)
                else:
                    index = k-j-1
                    id_s = k - j - 1
                    aut.new_edge(end_state, begin_state , cond)
                    for l in range(begin_state, end_state):
                        curr_state += 1
                        aut.new_edge(l,l+1, cond_id[index])
                        index += 1
                        id_sub_state[l] = (z,id_s)
                        id_s += 1
                    id_sub_state[end_state] = (z,id_s)


        curr_state+=1
    return aut,id_last_states,id_sub_state

   
def constraints_product_all(trailaut, faut, dict_id_state, info_states):
    p = spot.product(trailaut, faut)
    si = spot.scc_info(p)
    if not si.is_useful_state(p.get_init_state_number()):
        return [], []
    ps = p.get_product_states()
    k = trailaut.num_states()
    K_real = len(trail)
    edge_constraints = [ [buddy.bddfalse for i in range(K_real-1)] for j in range(K_real)]
    loop_constraints = [ buddy.bddfalse ] * K_real
    prefix_constraints = [ buddy.bddfalse] * (K_real -1)
    for scc in range(si.scc_count()):
        if not si.is_useful_scc(scc):
            continue
        for s in si.states_of(scc):
            step = ps[s][0]
            if step in dict_id_state.keys():
                for e in p.out(s):
                    #print(e.dst)
                    if si.is_accepting_scc(si.scc_of(e.dst)):
                            #print(e.src, e.dst, ps[e.dst][0])
                        print(step,dict_id_state[step], ps[e.dst][0])#,dict_id_state[ps[e.dst][0]]) dict_id_state[ps[e.dst][0]]]
                        loop_constraints[dict_id_state[step]] |= e.cond
            else:
                for e in p.out(s):
                    # print(step, ps[s], ps[e.dst], si.is_useful_state(e.dst))
                    if si.is_useful_state(e.dst):
                        if step < K_real - 1 :
                            prefix_constraints[step] |= e.cond
                        elif step > K_real - 1:
                            a,r = info_states[step]
                            edge_constraints[a][r] |= e.cond
                    
    for i in range(K_real - 1):
        const = prefix_constraints[i]
        for a in  range(i+1,K_real ):
            edge_constraints[a][i] |= const
        
    return ([str(spot.bdd_to_formula(c, p.get_dict())) for c in loop_constraints], 
            [[str(spot.bdd_to_formula(c, p.get_dict())) for c in i] for i in edge_constraints],
            #[str(spot.bdd_to_formula(c, p.get_dict())) for c in edge_constraints],
            [str(spot.bdd_to_formula(c, p.get_dict())) for c in prefix_constraints])
          #  [[str(spot.bdd_to_formula(c, p.get_dict())) for c in edge_constraints_per_aut[i]] for i in range(K_real-1)])
                                              
    


In [94]:
def trail_to_automaton_once(trail, bdddict, id_loop):
    aut = spot.make_twa_graph(bdddict)
    k = len(trail)
    id_sub_state  = dict()
    id_last_states = dict()
    cond_id = [buddy.bddtrue for i in range(k)]
    curr_state = 0
    aut.new_states(k)
    for i, step in enumerate(trail):
        cond = buddy.bddtrue
        for (ap, val) in step:
            if val == 0: 
                continue
            var = aut.register_ap(ap)
            if val == 1:
                cond &= buddy.bdd_ithvar(var)
            else:
                cond &= buddy.bdd_nithvar(var)
        cond_id[i] = cond
        if i != k - 1:
            aut.new_edge(i, i+1, cond)
        else:
            aut.new_edge(i,id_loop,cond)


        curr_state+=1
    return aut,id_last_states,id_sub_state

In [95]:
   
def constraints_product(trailaut, faut, dict_id_state, info_states):
    p = spot.product(trailaut, faut)
    si = spot.scc_info(p)
    if not si.is_useful_state(p.get_init_state_number()):
        return [], []
    ps = p.get_product_states()
    k = trailaut.num_states()
    K_real = len(trail)
    edge_constraints = [ [buddy.bddfalse for i in range(K_real-1)] for j in range(K_real)]
    loop_constraints = [ buddy.bddfalse ] * K_real
    prefix_constraints = [ buddy.bddfalse] * (K_real -1)
    for scc in range(si.scc_count()):
        if not si.is_useful_scc(scc):
            continue
        for s in si.states_of(scc):
            step = ps[s][0]
            if step in dict_id_state.keys():
                for e in p.out(s):
                    #print(e.dst)
                    if si.is_accepting_scc(si.scc_of(e.dst)):
                            #print(e.src, e.dst, ps[e.dst][0])
                        print(step,dict_id_state[step], ps[e.dst][0])#,dict_id_state[ps[e.dst][0]]) dict_id_state[ps[e.dst][0]]]
                        loop_constraints[dict_id_state[step]] |= e.cond
            else:
                for e in p.out(s):
                    # print(step, ps[s], ps[e.dst], si.is_useful_state(e.dst))
                    if si.is_useful_state(e.dst):
                        if step < K_real - 1 :
                            prefix_constraints[step] |= e.cond
                        elif step > K_real - 1:
                            a,r = info_states[step]
                            edge_constraints[a][r] |= e.cond
                    
    for i in range(K_real - 1):
        const = prefix_constraints[i]
        for a in  range(i+1,K_real ):
            edge_constraints[a][i] |= const
        
    return ([str(spot.bdd_to_formula(c, p.get_dict())) for c in loop_constraints], 
            [[str(spot.bdd_to_formula(c, p.get_dict())) for c in i] for i in edge_constraints],
            #[str(spot.bdd_to_formula(c, p.get_dict())) for c in edge_constraints],
            [str(spot.bdd_to_formula(c, p.get_dict())) for c in prefix_constraints])
          #  [[str(spot.bdd_to_formula(c, p.get_dict())) for c in edge_constraints_per_aut[i]] for i in range(K_real-1)])
                                              
    

In [96]:
def constraints_product(trailaut, faut, dict_id_state, list_sub_states_id):
    p = spot.product(trailaut, faut)
    si = spot.scc_info(p)
    if not si.is_useful_state(p.get_init_state_number()):
        return [], []
    ps = p.get_product_states()
    k = trailaut.num_states()
    K_real = len(trail)
    edge_constraints = [buddy.bddfalse] * (k) 
    for scc in range(si.scc_count()):
        if not si.is_useful_scc(scc):
            continue
        for s in si.states_of(scc):
            step = ps[s][0]
            if step == k - 1:
                for e in p.out(s):
                    #print(e.dst)
                    if si.is_accepting_scc(si.scc_of(e.dst)):
                        #print(e.src, e.dst, ps[e.dst][0])
                        #print(ps[e.dst][0], K_real)#,dict_id_state[ps[e.dst][0]])
                        edge_constraints[step] |= e.cond
                    else:
                        print("not acceptin scc!!", str(spot.bdd_to_formula(e.cond,p.get_dict())))
            else:
                for e in p.out(s):
                    # print(step, ps[s], ps[e.dst], si.is_useful_state(e.dst))
                    if si.is_useful_state(e.dst):
                        edge_constraints[step] |= e.cond
                    else:
                        print("not useful!!")
                        print(e.cond)
      
        
    return ([str(spot.bdd_to_formula(c, p.get_dict())) for c in edge_constraints])
                                              
    

In [97]:
trail = [[("a", 1), ("b", -1)], 
         [("a", 0), ("b", -1)], 
         [("a", 0), ("b", 1)]]

faut = spot.translate("FG a & X !b")
#faut = spot.translate('!( G ((!(a & b) & !(a & !b)) & !(b & !a)))')
aut,dict_id_state, big_dic = trail_to_automaton_all(trail, faut.get_dict())
display_inline(aut.show(".k"))
aut,dict_id_state, big_dic = trail_to_automaton_once(trail, faut.get_dict(),0)
p = spot.product(aut, faut)
display_inline(aut.show(".s"), faut, p.show(".s"))
constraints_product(aut,faut, dict_id_state, big_dic)



not acceptin scc!! b


['a & !b', '!b', 'a & b']

In [98]:
trail = [[("a", -1), ("b", -1), ("c",-1)], 
         [("a", -1), ("b", -1), ("c",-1)]]

faut = spot.translate("!( G ((!(a & b) & !(a & c)) & !(b & c)))")
aut,dict_id_state, big_dic = trail_to_automaton_all(trail, faut.get_dict())
p = spot.product(aut, faut)
display_inline(aut.show(".s"), faut.show(".s"), p.show(".s"))
constraints_product(aut,faut, dict_id_state, big_dic)



([], [])